In [1]:
import xarray as xr

In [ ]:
# Loop over datasets and interpolate onto z

### Interp vertical resolution of datasets from ds to a single coordinate for comparison 
### OUTCOME: original datasets in ds

## Get var 
# var = 'T'  ## Commented out as set above - search for #first_assign

## Set up obs dataset
# ds_obs = ds_obs ## Commented out as set above
# ds_obs = ds_obs.rename({'z': 'z_obs'})

## Set-up z interp
z_interp = np.arange(140,0,-1)

ds_interp_temp = xr.Dataset()
ds_interp = []
ds_ctrl_delta = [] ## For final T profile of CTRL + delta

for i in range(12):
    print("Calculating month", i+1)

    ## Set up temp dataset replacing lev with z as indexed dimension
    ds_nonrr_temp = ds_nonrr.isel(time=i).swap_dims({'lev': 'z'})
    ds_rr_temp = ds_rr.isel(time=i).swap_dims({'lev': 'z'})
    ds_mvg_c_temp = ds_mvg_c.isel(time=i).swap_dims({'lev':'z'})

    ## Set up z values for interpolation from mvg ctrl 
    # z_interp = ds_mvg_c_temp['z']

    ## Interpolate Non-RR and RR to z
    ds_nonrr_temp = ds_nonrr_temp.interp(z=z_interp)
    ds_rr_temp = ds_rr_temp.interp(z=z_interp)
    ds_mvg_c_temp = ds_mvg_c_temp.interp(z=z_interp)
    
    ## Create delta dataset to have correct dimensions
    ds_delta_temp = ds_rr_temp*0
    ctrl_delta = ds_rr_temp*0
    # ds_delta_temp['delta'] = ds_rr_temp['T'].isel(time=i)*0


    ## Calculate delta between Non-RR and RR and assign to delta dataset
    ds_delta_temp['delta'] = ds_rr_temp['T'] - ds_nonrr_temp['T']


    ## Calculate ds_ctrl_delta: ctrl values + delta from Non-RR and RR
    ctrl_delta['T'] = ds_mvg_c_temp['T'] + ds_delta_temp['delta']

    ## Append delta calculation for time index i to ds_delta list
    ## - will concatenate to full ds once loop completes
    ds_delta.append(ds_delta_temp)

    ## Append ctrl_delta calculation for time index i to ds_ctrl_delta list
    ## - will concatenate to full ds once loop completes
    ds_ctrl_delta.append(ctrl_delta)


## Concatenate all deltas together to a time ordered dataset, scroll to cell below to inspect ds_delta
ds_delta = xr.concat(ds_delta,dim="time")
ds_ctrl_delta = xr.concat(ds_ctrl_delta,dim="time")

#################################################################
#### STAPLE CODE USEFUL FOR COPYING #############################
#################################################################

#     print("Calculating month", i+1)
#     ## Isolate time for dataset -> create new variable delta to assign to
#     ## -> isolate variable from dataset -> Interpolate data on obs z
#     dataset_temp = dataset.isel(time=i).swap_dims({'lev': 'z'}).interp(z=z_interp)
#     dataset_temp['delta'] = dataset_temp[var]*0
#     dataset_temp['delta'] = (
#         dataset[var].isel(time=i).swap_dims({'lev': 'z'}).interp(z=z_interp) - ds_obs[var].isel(time=i)
#     )
#     ds_temp1.append(dataset_temp) ## Add delta for time i to list ds_temp
# ds_temp1 = xr.concat(ds_temp1, dim='time') ## Concatente ds_temp to single list
# ds_temp2.append(ds_temp1)
# ds = ds_temp2


# ds = ds[0:-1] ## to remove obs from dataset if included - should be last in list

    
# ### Calculate deltas and write to file
# for i in range(12):
#     print("Calculating delta for month", i+1)

#     ## Create dimension z and remove lev
#     ds_nonrr_temp = ds_nonrr.isel(time=i)
#     ds_nonrr_temp_z = ds_nonrr_temp.swap_dims({'lev':'z'})
#     ds_rr_temp = ds_rr.isel(time=i)
#     ds_rr_temp_z = ds_rr_temp.swap_dims({'lev':'z'})

#     ## Interpolate RR z onto Non-RR z
#     ds_rr_temp_z = ds_rr_temp_z.interp(z = ds_nonrr_temp_z['z'])

#     ## Calculate delta
#     delta = ds_rr_temp_z - ds_nonrr_temp_z

#     ## Write delta to file
#     delta.to_netcdf(data_path + file_str + "delta_" + str(delta['time'].values) + ".nc")
# z_interp.values
